# Cats and dogs classifier



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

D:\Users\Евгений Артёменко\PycharmProjects\Projects\Machine learning\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Этап 1: Подготовка данных

In [248]:
class CatDogsDataset(torch.utils.data.Dataset):
    def __init__(self, path_dir1:str, path_dir2:str):
        super().__init__()
        
        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2
        
        self.dir1_list = sorted(os.listdir(path_dir1))
        self.dir2_list = sorted(os.listdir(path_dir2))
        
    def __len__(self):
        return len(self.dir1_list) + len(self.dir2_list)
    
    def __getitem__(self, idx):
        
        if idx < len(self.dir1_list):
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
        else:
            class_id = 1
            idx -= len(self.dir1_list)
            img_path = os.path.join(self.path_dir2, self.dir2_list[idx])
        
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        
        img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))
        
        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)
        
        return {'img': t_img, 'label': t_class_id}

In [249]:
train_dogs_path = 'training_set\dogs'
train_cats_path = 'training_set\cats'

test_dogs_path = 'test_set\dogs'
test_cats_path = 'test_set\cats'

train_ds = CatDogsDataset(train_dogs_path, train_cats_path)
test_ds = CatDogsDataset(test_dogs_path, test_cats_path)

In [250]:
# train_ds[0]['img']
# len(train_ds)
# len(test_ds)

## Этап 2: DataLoader

In [251]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(
    train_ds, batch_size=batch_size, shuffle=True, drop_last=True
)

test_loader = torch.utils.data.DataLoader(
    test_ds, batch_size=batch_size, drop_last=False
)

## Этап 3: Архитектура нейроннной сети

In [252]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.activation = nn.LeakyReLU(0.2)
        
        self.maxpool = nn.MaxPool2d(2, 2)
        self.conv0 = nn.Conv2d(3, 128, 3, stride=1, padding=0)
        self.conv1 = nn.Conv2d(128, 128, 3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(128, 128, 3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(128, 256, 3, stride=1, padding=0)
        
        self.adaptivepool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(256, 20)
        self.linear2 = nn.Linear(20, 2)
        
        
    def forward(self, x):
        out = self.conv0(x)
        out = self.activation(out)
        out = self.maxpool(out)

        out = self.conv1(out)
        out = self.activation(out)
        out = self.maxpool(out)

        out = self.conv2(out)
        out = self.activation(out)
        out = self.maxpool(out)

        out = self.conv3(out)
        out = self.activation(out)

        out = self.adaptivepool(out)
        out = self.flatten(out)
        out = self.linear1(out)
        out = self.activation(out)
        out = self.linear2(out)
                
        return out

In [253]:
model = ConvNet()
model

ConvNet(
  (activation): LeakyReLU(negative_slope=0.2)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (adaptivepool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=256, out_features=20, bias=True)
  (linear2): Linear(in_features=20, out_features=2, bias=True)
)

In [254]:
for sample in train_loader:
    img = sample['img']
    img_id = sample['label']
    model(img)
    break

In [256]:
def accuracy(pred, label):
    answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
    return answer.mean()

In [257]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

## Этап 4: Обучение нейронной сети

In [258]:
epochs = 10

for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm(train_loader)):
        img, label = sample['img'], sample['label']
        optimizer.zero_grad()
        
        label = F.one_hot(label, 2).float()
        pred = model(img)
        
        loss = loss_func(pred,label)
        loss.backward()
        loss_val += loss.item()
        
        optimizer.step()
        
        acc_val += accuracy(pred, label)
        
    pbar.set_description(f'loss: {loss.item()} \ accuracy: {accuracy(pred, label)}')
    print(loss_val/len(train_loader))
    print(acc_val/len(train_loader))
        

  0%|                                                                                                                                          | 0/500 [00:00<?, ?it/s]D:\Users\Евгений Артёменко\AppData\Local\Temp\ipykernel_29396\1416373963.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [10:02<00:00,  1.21s/it]


0.6970200011730194
0.523


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [26:43<00:00,  3.21s/it]


0.6867218539118767
0.54875


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [10:36<00:00,  1.27s/it]


...

0.5203351802825927
0.746875


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [06:35<00:00,  1.27it/s]


0.48964941254258154
0.7665


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [07:32<00:00,  1.10it/s]

0.4531156561374664
0.790875


## Этап 5: Тестирование

In [ ]:
loss_val = 0
acc_val = 0
for sample in (pbar := tqdm(test_loader)):
    with torch.no_grad():
        img, label = sample['img'], sample['label']

        label = F.one_hot(label, 2).float()
        pred = model(img)

        loss = loss_func(pred, label)
        loss_item = loss.item()
        loss_val += loss_item

        acc_current = accuracy(pred, label)
        acc_val += acc_current

    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
print(loss_val/len(train_loader))
print(acc_val/len(train_loader))